In [15]:
import mysql.connector
from pprint import pprint
import sys
import pandas as pd
import re

In [16]:
class DB:
    def __init__(self,host,user,password):  
        try:
            self.db_conn = mysql.connector.connect(host=host,user=user,password=password)        
        except Exception as e:
            print(e)
            sys.exit(0)

    def create_db(self):
        try:    
            query_delete_if_db_exists = "DROP DATABASE IF EXISTS RandomX"
            query_create_db = "CREATE DATABASE RandomX"
            query_use_db = "USE RandomX"
            db_cursor = self.db_conn.cursor()        
            db_cursor.execute(query_delete_if_db_exists) 
            db_cursor.execute(query_create_db) 
            db_cursor.execute(query_use_db) 
        except Exception as e:
            print(e)
            sys.exit(0)


    def create_schema_for_tables(self):
        try:
            query_user_table = "CREATE TABLE users (username VARCHAR(100) NOT NULL ,gender enum('m','f') NOT NULL,age INT NOT NULL,PRIMARY KEY (username))"
            query_movie_table = "CREATE TABLE movies (movie_id VARCHAR(100) NOT NULL ,movie_title VARCHAR(100) NOT NULL,audience_rating DECIMAL(3,2) NOT NULL,PRIMARY KEY (movie_id))"
            query_movie_review_table = "CREATE TABLE movie_reviews (review_id INT NOT NULL AUTO_INCREMENT,movie_id VARCHAR(100) NOT NULL ,username VARCHAR(100) NOT NULL,movie_rev VARCHAR(1000) NOT NULL,PRIMARY KEY (review_id),FOREIGN KEY (username) REFERENCES users(username), FOREIGN KEY (movie_id) REFERENCES movies(movie_id))"
            db_cursor = self.db_conn.cursor()        
            db_cursor.execute(query_user_table)
            db_cursor.execute(query_movie_table)
            db_cursor.execute(query_movie_review_table)
        except Exception as e:
            print(e)
            sys.exit(0)      

    def insert_user_data(self, user_data):
        try:
            db_cursor = self.db_conn.cursor()
            query = "INSERT INTO users (username, gender, age) VALUES (%s, %s, %s)"
            db_cursor.executemany(query, user_data)
            print(str(db_cursor.rowcount)+" row(s) inserted.")
            self.db_conn.commit()
        except Exception as e:
            print(e)
            sys.exit(0)

    def insert_movie_data(self, movie_data):
        try:
            db_cursor = self.db_conn.cursor()            
            query = "INSERT INTO movies (movie_id, movie_title, audience_rating) VALUES (%s, %s, %s)"
            db_cursor.executemany(query, movie_data)
            print(str(db_cursor.rowcount)+" row(s) inserted.")
            self.db_conn.commit()
        except Exception as e:
            print(e)
            sys.exit(0)

    def insert_movie_review_data(self, movie_review_data):
        try:
            db_cursor = self.db_conn.cursor()            
            query = "INSERT INTO movie_reviews (movie_id, username, movie_rev) VALUES (%s, %s, %s)"
            db_cursor.executemany(query, movie_review_data)
            print(str(db_cursor.rowcount)+" row(s) inserted.")
            self.db_conn.commit()
        except Exception as e:
            print(e)
            sys.exit(0)

    def create_sentiment_column(self):
        try:
            db_cursor = self.db_conn.cursor()            
            query = "ALTER TABLE movie_reviews ADD COLUMN sentiment enum('positive','negative','neutral') NOT NULL"
            db_cursor.execute(query)            
        except Exception as e:
            print(e)
            sys.exit(0)    

    def update_sentiment_value(self, positive_words, negative_words):
        try:            
            db_cursor = self.db_conn.cursor()
            query = "SELECT * FROM movie_reviews"
            db_cursor.execute(query)                  
            data = db_cursor.fetchall()  
            column_names = [column[0] for column in db_cursor.description] 
            reviews = []
            
            for row in data:
                reviews.append(dict(zip(column_names, row)))
            
            for review in reviews:
                
                positive_count = 0
                negative_count = 0
                exp = re.compile('[A-Za-z]+')
                words_in_review = list(exp.findall(review['movie_rev'].lower()))   
                
                for word in words_in_review:
                    if word in positive_words:
                        positive_count+=1
                    if word in negative_words:
                        negative_count+=1
                        
                if positive_count > negative_count:
                    sentiment = "positive"
                elif positive_count < negative_count:
                    sentiment = "negative"
                else:
                    sentiment = "neutral"
                    
                query = "UPDATE movie_reviews SET sentiment = %s WHERE review_id = %s"
                db_cursor.execute(query, (sentiment, review['review_id']))
                self.db_conn.commit()                            
                
        except Exception as e:
            print(e)
            sys.exit(0)  
    
    def find_movies(self, limit, rating, positive_sentiment):
        try:
            db_cursor = self.db_conn.cursor()
            query = "SELECT unsorted_table.movie_title FROM (SELECT movies.*, IFNULL(movies_with_positive_sentiment_count.positive_sentiment_count, 0) as positive_sentiment_value FROM movies LEFT OUTER JOIN (SELECT movies.*, COUNT(*) as positive_sentiment_count FROM movies NATURAL JOIN movie_reviews WHERE movie_reviews.sentiment = 'positive' GROUP BY movies.movie_id) as movies_with_positive_sentiment_count ON movies_with_positive_sentiment_count.movie_id = movies.movie_id WHERE movies.audience_rating > %s HAVING positive_sentiment_value >= %s) as unsorted_table ORDER BY unsorted_table.audience_rating DESC LIMIT %s"
            db_cursor.execute(query, (rating, positive_sentiment, limit))
            data = db_cursor.fetchall()
            column_names = [column[0] for column in db_cursor.description] 
            self.print_for_select(column_names, data)
        except Exception as e:
            print(e)
            sys.exit(0) 

    def print_for_select(self, column_names, data):
        for row in data:
            pprint(dict(zip(column_names, row)))
            print()    

    def close(self):
        try:
            self.db_conn.close()
        except Exception as e:
            print(e)
            sys.exit(0)        

In [17]:
db = DB("localhost","newuser","newpass")

### Part A

In [18]:
db.create_db()
db.create_schema_for_tables()

### Part B

In [19]:
try:
    user_data_raw = pd.read_excel('userdata.xls')
except Exception as e:
    print(e)
    sys.exit(0)

user_data = []
for index in range(len(user_data_raw)):
    user_data.append((user_data_raw['username'][index], user_data_raw['gender'][index], int(user_data_raw['age'][index])))

db.insert_user_data(user_data)


try:
    movie_data_raw = pd.read_excel('movie_info.xlsx')
except Exception as e:
    print(e)
    sys.exit(0)

movie_data = []
for index in range(len(movie_data_raw)):
    movie_data.append((movie_data_raw['movie_id'][index], movie_data_raw['movie_title'][index], float(movie_data_raw['audience_rating'][index])))

db.insert_movie_data(movie_data)

try:
    movie_review_data_raw = pd.read_excel('movie_reviews.xls')
except Exception as e:
    print(e)
    sys.exit(0)

movie_review_data = []
for index in range(len(movie_review_data_raw)):
    movie_review_data.append((movie_review_data_raw['movie_id'][index], movie_review_data_raw['user_name'][index], movie_review_data_raw['movie_rev'][index]))    

db.insert_movie_review_data(movie_review_data)

20 row(s) inserted.
100 row(s) inserted.
500 row(s) inserted.


### Part C

In [20]:
positive_words_file = open('positive-words.txt')
positive_words = set([line.rstrip().lower() for line in positive_words_file.readlines()])
negative_words_file = open('negative-words.txt')
negative_words = set([line.rstrip().lower() for line in negative_words_file.readlines()])

db.create_sentiment_column()
db.update_sentiment_value(positive_words, negative_words)

### Part D

In [21]:
db.find_movies(5, 3.5, 2)

{'movie_title': 'The Lord of the Rings: The Fellowship of the Ring'}

{'movie_title': 'Room'}

{'movie_title': 'The Man Who Shot Liberty Valance'}

{'movie_title': 'Duck Soup'}

{'movie_title': 'Run Lola Run'}

